## LSTM 尼采 文本生成

In [5]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path, encoding = 'utf8').read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [6]:
maxlen = 60
step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Number of sequences: 200278
Unique characters: 57
Vectorization...


In [28]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [29]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### softmax temperature

In [30]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
			

epoch 1
Epoch 1/1
200278/200278 [==============================] - 206s 1ms/step - loss: 2.0126
--- Generating with seed: "of the people, who exist for service and
general utility, an"
------ temperature: 0.2
of the people, who exist for service and
general utility, and his the moral and and that the respection the more the despection that the moral the the the man in the man in the more the more the most of the sense of the person the senses the sense of the sense of the man in the man in the sense that the the man in the man in the man in the sense of the sense of the sense the man in the man in the beto more the more that the sense of the men in the senses t------ temperature: 0.5
beto more the more that the sense of the men in the senses that he one dound truth of more that the sense of the simmanity the sense of that the free things of the hever to the instinction that on the more
prece that the but in the somity of the but as that it is the laster and conto hand in"man the man, t

C:\Users\sy3445\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


head of the wast to power of the state of men and acts in not the condition of the most taste the will of the former to his man in the fact that ------ temperature: 1.0
st taste the will of the former to his man in the fact that a mean by the main thoughtly and shortatian. justicaty to doing a know of a stopls
pacts that they is state in the taveios will--be begode something to uto"mann as we has withone (the jestines as if these the reimanists, and poptumentmy in literaters: is the word, such as he substious was much with restrainess, the devalocism.--who is revolicing to in empersise man, the deces of us.

entiobll even------ temperature: 1.2
olicing to in empersise man, the deces of us.

entiobll eventures
did eken to his nothoin life or perhavs glosula, is, their inventifly the sposists upon and is is the vationanittiation" in
or unwill, who is revulief the restraining onfest is fasmiysments of attaming man is le ladpabioring with troth; as muscuoust
do highc furchic moreness, atte